# LTSM SIMPLE



## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.



In [20]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader
from math import sqrt


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/01.Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data('non-secuencial')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total_for_non_serial.csv
(30555, 34)
________________________________________________________________________________


In [21]:
data_consumo.columns

Index(['fecha', 'PVPC_DEF', 'PVPC_2_PED_NOC', 'PVPC_ELEC_NOC',
       'date_timestamp', 'Demanda', 'Eolica', 'Nuclear', 'Solar',
       'Solar_Fotovoltaica', 'Solar_Termica', 'Termica_Renovable', 'Prevista',
       'Programada', 'date_day', 'Brent_price', 'Holiday',
       'Precio de Regulación Secundaria subir',
       'Precio de Regulación Secundaria bajar', 'Precio mercado SPOT Diario_x',
       'Demanda real', 'Generación prevista Solar',
       'Saldo total interconexiones programa p48',
       'Generación programada P48 Exportación Portugal',
       'Generación programada P48 Exportación Francia',
       'Generación programada P48 Importación Portugal',
       'Generación programada P48 Importación Francia', 'Precio SPOT PT',
       'Precio SPOT FR', 'PVPC-target', 'PVPC-1h', 'PVPC-2h', 'PVPC-1d',
       'PVPC-1w'],
      dtype='object')

## Preparación de los datos

Sobre el tema de los datos. El objetivo es predecir el precio a d+1.

In [22]:
x_data = esios_controller.get_data_real_time()
y_data = esios_controller.get_target_data()

x_data.rename(columns={'date_timestamp':'timestamp'},  inplace=True)
y_data.rename(columns={'date_timestamp':'timestamp'},  inplace=True)
x_data.head()

,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR,PVPC-1h,PVPC-2h,PVPC-1d,PVPC-1w
0,117.77,69.49,71.88,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,47.48,0.0,61.18,49.13,56.65,25305.0,267.6,182.4,-606.6,-351.4,1232.1,1200.0,56.65,38.56,0.00,0.00,0.0,0.0
1,117.09,68.21,65.57,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,47.48,0.0,59.59,49.13,53.50,23959.0,195.7,-269.5,-1112.5,-351.4,1232.1,1200.0,53.50,31.27,117.77,0.00,0.0,0.0
2,114.59,65.34,62.53,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,-351.4,1232.1,1200.0,49.69,30.66,117.09,117.77,0.0,0.0
3,116.60,66.53,63.40,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,-351.4,1232.1,1200.0,49.10,29.14,114.59,117.09,0.0,0.0
4,122.07,71.03,67.37,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,-351.4,1232.1,1200.0,51.25,29.40,116.60,114.59,0.0,0.0


In [23]:
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

#Convert NaN values to 0:
x_train = np.nan_to_num(x_train)
x_valid = np.nan_to_num(x_valid)
y_train = np.nan_to_num(y_train)
y_valid = np.nan_to_num(y_valid)

print('Xtrain_dim:', x_train.shape)
print('Xvalid_dim:', x_valid.shape)
print('Ytrain_dim:', y_train.shape)
print('Yvalid_dim:', y_valid.shape)

Xtrain_dim: (20471, 30)
Xvalid_dim: (10084, 30)
Ytrain_dim: (20471, 1)
Yvalid_dim: (10084, 1)


In [24]:
print(y_train)

[[123.3 ]
 [121.98]
 [126.7 ]
 ...
 [121.08]
 [121.31]
 [120.97]]


In [0]:
def rmse (y_true, y_pred):
  return sqrt(mean_squared_error(y_pred, y_true))

## Modelo

In [0]:
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import SGD
from keras.regularizers import l2

batch_size = 16
num_steps = 20
num_var = x_train.shape[1]
hidden_size = 10
output_size = 1

In [0]:
# split a sequence into samples
def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
        break
      # gather input and output parts of the pattern
      seq_x = sequence[i:end_ix,:]
      X.append(seq_x)

    return np.array(X)

In [28]:
scaler = MinMaxScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)

x_train_data = split_sequence(x_train_est, num_steps)
y_train_data = y_train_est[num_steps:]
x_valid_data = split_sequence(x_valid, num_steps)
y_valid_data = y_valid[num_steps:]

print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (20451, 20, 30) (20451, 1)
Test: (10064, 20, 30) (10064, 1)


In [30]:
# define model
model = Sequential()
model.add(LSTM(hidden_size, 
             activation='relu', 
             return_sequences=True, 
             kernel_initializer='zeros',
             input_shape=(num_steps,num_var),
             kernel_regularizer=l2(1e-2)))
model.add(Dropout(0.4))
model.add(LSTM(hidden_size, 
             activation='relu',
             kernel_initializer='zeros',
             kernel_regularizer=l2(1e-2)))
model.add(Dropout(0.5))
model.add(Dense(1,
             activation='linear',
             kernel_regularizer=l2(1e-2)))

# compile mode
opt = SGD(lr=0.1, momentum=0.9, decay= 1e-2, nesterov=True)
model.compile(optimizer=opt, loss='mse', metrics=['mse',rmse,'mae','mape'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20, 10)            1640      
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10)                840       
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 2,491
Trainable params: 2,491
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
history_train = model.fit(x_train_data, y_train_data, 
                        epochs=100, 
                        batch_size=batch_size, 
                        shuffle=False,
                        verbose=1)

# summarize history for loss, MAPE in a different scale than the others
plt.plot(history_train.history['mean_squared_error'])
plt.plot(history_train.history['mean_absolute_error'])
plt.plot(history_train.history['rmse'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MSE','MAE','RMSE'], loc='upper left')
plt.show()

plt.plot(history_train.history['mean_absolute_percentage_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MAPE'], loc='upper left')
plt.show()

W0827 19:28:04.864154 140211643352960 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0827 19:28:05.906802 140211643352960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
20451/20451 [==============================] - 83s 4ms/step - loss: 0.0091 - mean_squared_error: 0.0086 - rmse: 0.0688 - mean_absolute_error: 0.0688 - mean_absolute_percentage_error: 12395.5848
Epoch 2/100
20451/20451 [==============================] - 78s 4ms/step - loss: 0.0092 - mean_squared_error: 0.0092 - rmse: 0.0704 - mean_absolute_error: 0.0704 - mean_absolute_percentage_error: 11199.7520
Epoch 3/100
20451/20451 [==============================] - 78s 4ms/step - loss: 0.0095 - mean_squared_error: 0.0095 - rmse: 0.0714 - mean_absolute_error: 0.0714 - mean_absolute_percentage_error: 12374.0763
Epoch 4/100
20451/20451 [==============================] - 77s 4ms/step - loss: 0.0098 - mean_squared_error: 0.0098 - rmse: 0.0726 - mean_absolute_error: 0.0726 - mean_absolute_percentage_error: 13172.7855
Epoch 5/100
12624/20451 [=================>............] - ETA: 29s - loss: 0.0107 - mean_squared_error: 0.0107 - rmse: 0.0788 - mean_absolute_error: 0.0788 - mean_absolute_per

KeyboardInterrupt: ignored